In [3]:
import pandas as pd
import numpy as np
from eep153_tools.sheets import read_sheets
Ethiopia_Data = '1qOZi1FqAS89x461pCIgwRaAfpBw5aUp7kBbp0tQKI_E'
# Change 'Ethiopia_Data' to key of your own sheet in Sheets, above
x = read_sheets(Ethiopia_Data,sheet = 'Food Expenditures (2011-12)')
x = x.set_index(['i','t','m','j']).squeeze()


# Now prices
p = read_sheets(Ethiopia_Data,sheet='Food Prices (2011-12)').set_index(['t','m','j','u'])

# Compute medians of prices for particular time, place and unit
p = p.groupby(['t','m','j','u']).median()

# Just keep metric units
p = p.xs('Kg',level="u").squeeze().unstack('j')

# Drop prices for goods we don't have expenditures for
# p = p[x.index.levels[-1]].T
p =  p.dropna()

# Household characteristics
d = read_sheets(Ethiopia_Data,sheet="Household Characteristics")
d.columns.name = 'k'

# Fill blanks with zeros
d = d.replace(np.nan,0)

# Expenditures x may have duplicate columns
x = x.T.groupby(['i','t','m','j']).sum()
x = x.replace(0,np.nan) # Replace zeros with missing

# Take logs of expenditures; call this y
y = np.log(x)

d.set_index(['i','t','m'],inplace=True)

ModuleNotFoundError: No module named 'eep153_tools'

In [4]:
d.groupby('t').mean()

NameError: name 'd' is not defined

In [5]:
from cfe import Regression

result = Regression(y=y, d=d)

ModuleNotFoundError: No module named 'cfe'